<a href="https://colab.research.google.com/github/AlexH20/Tukey/blob/master/TukeyEmpirical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive 
import scipy.stats as stats
import scipy.optimize as opt

drive.mount("/content/gdrive")

def tukeyroot(x):

  F1 = -S+((3*np.exp(x[0]**2/(2-6*x[1]))+np.exp(9*x[0]**2/(2-6*x[1]))-3*np.exp(2*x[0]**2/(1-3*x[1]))-1)/(x[0]**3*(1-3*x[1])**(1/2))-
    3*((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))*((1-2*np.exp(x[0]**2/(2-4*x[1]))+np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2)))+
    2*((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))**3)/((((1-2*np.exp(x[0]**2/(2-4*x[1]))+np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2)))-
    ((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))**2)**(3/2))


  F2 = -K+(((np.exp(8*x[0]**2/(1-4*x[1]))*(1+6*np.exp(6*x[0]**2/(4*x[1]-1))+np.exp(8*x[0]**2/(4*x[1]-1))-4*np.exp(7*x[0]**2/(8*x[1]-2))-4*np.exp(15*x[0]**2/(8*x[1]-2))))/(x[0]**4*(1-4*x[1])**(1/2)))-
    4*((3*np.exp(x[0]**2/(2-6*x[1]))+np.exp(9*x[0]**2/(2-6*x[1]))-3*np.exp(2*x[0]**2/(1-3*x[1]))-1)/(x[0]**3*(1-3*x[1])**(1/2)))*((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))-
    3*((1-2*np.exp(x[0]**2/(2-4*x[1]))+np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2)))**2+
    12*((1-2*np.exp(x[0]**2/(2-4*x[1]))+np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2)))*((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))**2-
    6*((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))**4)/((((1-2*np.exp(x[0]**2/(2-4*x[1]))+np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2)))-((np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2)))**2)**2)
    
  return [F1, F2]




#Read data to Matlab workspace

dax = pd.read_csv("gdrive/My Drive/dax2Y.csv", delimiter = ";", usecols = [1], header = None)
dax.dropna(inplace = True)
dax.rename(columns={dax.columns[0]: "Dax"}, inplace = True)
dax["Dax"] = [float(x.replace(",", ".")) for x in dax["Dax"]]

dax["Dax"] = dax["Dax"].pct_change(1)
dax.dropna(inplace = True)
print(dax)

S = stats.skew(dax["Dax"])
K = stats.kurtosis(dax["Dax"])
mean = np.mean(dax["Dax"])
std = np.std(dax["Dax"])

x0 = [1,0]
x = opt.fsolve(tukeyroot, x0)
print(K)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
          Dax
1    0.010927
2   -0.002059
3   -0.005094
4    0.005990
5   -0.004806
..        ...
517  0.000000
518  0.000000
519  0.002573
520 -0.005681
521  0.000000

[521 rows x 1 columns]
2.031753867900065


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


' \n %hist(sreturns);\n \n % calculate summary statistics of the data: mean, std, skew, kurtosis\n S=skewness(sreturns);\n K=kurtosis(sreturns);\n mean=mean(sreturns);\n std=std(sreturns);\n \n% specify obtained values S for skew and K for kurtosis in the auxiliary \'tukeyroot\' file \n% find \'g\' and \'h\' values by solving the system of nonlinear equations\n\n x0 = [1; 0];  % Make a starting guess at the solution\n[x,fval] = fsolve(@tukeyroot,x0) % Call solver\n x[0] % g\n x[1] % h\n \n % calculate two central moments of Tukey distribution as well as np.np.expectation and\n % variance\n \n E1 = (np.np.exp(x[0]**2/(2-2*x[1]))-1)/(x[0]*(1-x[1])**(1/2));\n E2 = (1-2*np.np.exp(x[0]**2/(2-4*x[1]))+np.np.exp(2*x[0]**2/(1-2*x[1])))/(x[0]**2*(1-2*x[1])**(1/2));\n\n tukey_mean = E1;\n tukey_variance = E2-E1**2;\n\n % calculate values necessary for the linear transformation\n \n a=std/tukey_variance;\n b=mean - a*tukey_mean;\n \n % specify linear transformation as following:\n \n R=randn(1,10